In [34]:
install.packages("readr")
library(readr)

install.packages("dplyr")
library(dplyr)

Warning message:
"pakiet 'readr' jest w użyciu i nie zostanie zainstalowany"
Warning message:
"pakiet 'dplyr' jest w użyciu i nie zostanie zainstalowany"


**Funkacja convert, która zamienia separator dziesiętny na przecinek oraz typ zmiennej numeric na intiger**

In [35]:
convert_commas_to_dots <- function(df) {
  # Zamiana przecinków na kropki w kolumnach typu
  #character tylko jeśli zawierają przecinek
  df[] <- lapply(df, function(x) {
    if (is.character(x) && any(grepl(",", x))) {
      x <- gsub(",", ".", x)
      num_x <- suppressWarnings(as.numeric(x))
      if (all(is.na(num_x) & !is.na(x))) {
        return(x) # Jeśli wszystkie wartości stają się NA, pozostaw jako tekst
      } else {
        return(num_x) # Konwersja na numeric, jeśli możliwe
      }
    } else {
      return(x)
    }
  })
  return(df)
}

convert_numeric_to_integer <- function(df) {

  #Zamiana wartości na integer, jeśli nie zawierają
  #kropki i nie są wszystkimi NA 

  df[] <- lapply(df, function(x) {
    if (is.numeric(x) && all(!grepl("\\.", as.character(x)), na.rm = TRUE)) { # nolint
      return(as.integer(x))
    } else {
      return(x)
    }
  })
  return(df)
}

In [36]:
convert <- function(df) {
  df <- convert_commas_to_dots(df)
  df <- convert_numeric_to_integer(df)
  return(df)
}

**Zaczytanie danych**

Błędy w zaczytywanych danych: 
- teams.csv ma podublowane rekordy, gdzie połowa z nuch jest niepoprawna. Należy usunąć wiersze, gdzie TM_TAG != TM_NAME
- teams.csv ma błędne ilości minut. Większość wartości w kolumnie MIN jest równa 200 (lub 225, 250, 300 w zależności od ilości dogrywek), gdyż zostały zsumowane z tabeli players.csv przez autora danych. Należy podzielić kolumnę MIN przez 5. 
- niespójność w players.csv. Część rekordów jest pomimo tego, że MIN = 0, co oznacza, że zawodnik nie wystąpił w meczu. Natomiast widać, że części rekordów nie ma, gdyż część zawodników ma mniejszą ilość rozegranych gier. Trzeba usunąć rekordy, gdzie MIN = 0, gdyż rodzi to problem przy zliczaniu ilości game played oraz liczeniu średniej. 

In [37]:
teams <- read_csv2("teams.csv")
players <- read_csv2("players.csv")

ℹ Using "','" as decimal and "'.'" as grouping mark. Use `read_delim()` for more control.

Rows: 1324 Columns: 133
── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
chr  (13): TM TAG, OPP TAG, TM NAME, OPP NAME, DATE, H/A, W/L, RIM PPS, MID ...
dbl (120): ROUND, MIN, PTS, 2PTM, 2PTA, 2PT%, 3PTM, 3PTA, 3PT%, FGM, FGA, FG...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
ℹ Using "','" as decimal and "'.'" as grouping mark. Use `read_delim()` for more control.

Rows: 7883 Columns: 85
── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
chr (14): TM TAG, NAME, ROLE, NAT, HEIGHT, TM NAME, OPP NAME, DATE, H/A, W/L...
dbl (71): AGE, ROUND, MIN, PTS, 2PTM, 2PTA, 3PTM, 3PTA, FGM, FGA, FTM, FTA, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or s

**Konwersja danych**

In [38]:
teams <- convert(teams)
glimpse(teams)

Rows: 1,324
Columns: 133
$ ROUND              <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, …
$ `TM TAG`           <chr> "CZB", "ASV", "MTA", "PAR", "EFE", "FCB", "BER", "B…
$ `OPP TAG`          <chr> "ASV", "CZB", "PAR", "MTA", "FCB", "EFE", "BAY", "B…
$ `TM NAME`          <chr> "Crvena Zvezda Meridianbet Belgrade", "LDLC ASVEL V…
$ `OPP NAME`         <chr> "LDLC ASVEL Villeurbanne", "Crvena Zvezda Meridianb…
$ DATE               <chr> "05.10.2023", "05.10.2023", "05.10.2023", "05.10.20…
$ `H/A`              <chr> "H", "A", "H", "A", "A", "H", "A", "H", "H", "A", "…
$ `W/L`              <chr> "W", "L", "W", "L", "L", "W", "L", "W", "L", "W", "…
$ MIN                <int> 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 2…
$ PTS                <int> 94, 73, 96, 81, 74, 91, 68, 80, 79, 82, 82, 85, 88,…
$ `2PTM`             <int> 16, 17, 27, 26, 22, 32, 16, 23, 23, 18, 18, 18, 19,…
$ `2PTA`             <int> 37, 29, 43, 39, 40, 42, 30, 36, 39, 39, 36, 34, 35,…
$ `2PT%`       

In [39]:
players <- convert(players)
glimpse(players)

Rows: 7,883
Columns: 85
$ `TM TAG`       <chr> "CZB", "CZB", "CZB", "CZB", "CZB", "CZB", "CZB", "CZB",…
$ NAME           <chr> "Adam Hanga", "Branko Lazic", "Dalibor Ilic", "Luka Mit…
$ ROLE           <chr> "F", "SG", "F", "PF", "PF", "C", "PG", "SG", "C", "F/PF…
$ NAT            <chr> "HUN", "SRB", "BIH", "SRB", "SRB", "USA", "SRB", "SRB",…
$ HEIGHT         <chr> "201", "195", "206", "206", "203", "213", "195", "191",…
$ AGE            <int> 35, 35, 24, 31, 38, 30, 37, 33, 34, 32, 33, 28, 30, 35,…
$ ROUND          <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ `TM NAME`      <chr> "Crvena Zvezda Meridianbet Belgrade", "Crvena Zvezda Me…
$ `OPP NAME`     <chr> "LDLC ASVEL Villeurbanne", "LDLC ASVEL Villeurbanne", "…
$ DATE           <chr> "05.10.2023", "05.10.2023", "05.10.2023", "05.10.2023",…
$ `H/A`          <chr> "H", "H", "H", "H", "H", "H", "H", "H", "H", "H", "H", …
$ `W/L`          <chr> "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", …
$ MIN           

In [40]:
# Podział danych na poszczególne etapy rozgrywek
# Dla zespołów
teams_regular <- teams %>% filter(ROUND <= 34)
teams_playin  <- teams %>% filter(ROUND >= 35 & ROUND <= 36)
teams_playoff <- teams %>% filter(ROUND >= 37 & ROUND <= 41)
teams_final4  <- teams %>% filter(ROUND >= 42 & ROUND <= 43)

# Dla zawodników
players_regular <- players %>% filter(ROUND <= 34)
players_playin  <- players %>% filter(ROUND >= 35 & ROUND <= 36)
players_playoff <- players %>% filter(ROUND >= 37 & ROUND <= 41)
players_final4  <- players %>% filter(ROUND >= 42 & ROUND <= 43)

In [41]:
print(teams_playin)

#Poniżej widać, że pojawiają się błędy w postaci niepotrzebnych recordów
#Należy usunąć wiersze, gdzie TM TAG jest niezgodny z TM NAME 
#np. 5 `TM TAG` = BAS, `TM NAME` = Maccabi Playtika Tel Aviv

# A tibble: 12 × 133
   ROUND `TM TAG` `OPP TAG` `TM NAME`   `OPP NAME` DATE  `H/A` `W/L`   MIN   PTS
   <int> <chr>    <chr>     <chr>       <chr>      <chr> <chr> <chr> <int> <int>
 1    35 BAS      MTA       Baskonia V… Maccabi P… 16.0… A     L       200    85
 2    35 MTA      BAS       Maccabi Pl… Baskonia … 16.0… H     W       200   113
 3    35 EFE      VIR       Anadolu Ef… Virtus Se… 16.0… H     L       200    64
 4    35 VIR      EFE       Virtus Seg… Anadolu E… 16.0… A     W       200    67
 5    35 BAS      MTA       Maccabi Pl… Baskonia … 16.0… A     L       200   113
 6    35 MTA      BAS       Baskonia V… Maccabi P… 16.0… H     W       200    85
 7    35 EFE      VIR       Virtus Seg… Anadolu E… 16.0… H     L       200    67
 8    35 VIR      EFE       Anadolu Ef… Virtus Se… 16.0… A     W       200    64
 9    36 BAS      VIR       Baskonia V… Virtus Se… 19.0… H     W       200    89
10    36 VIR      BAS       Virtus Seg… Baskonia … 19.0… A     L       200    77
11    3

**Czyszczenie danych teams**

In [42]:
tag_name_map <- tibble(
  `TM TAG` = c("CZB", "ASV", "MTA", "PAR", "EFE", "FCB", "BER", "BAY",
               "VIR", "ZAL", "MIL", "FEN", "OLY", "PAN", "BAS", "RMB",
               "ASM", "VAL"),
  `TM NAME` = c("Crvena Zvezda Meridianbet Belgrade",
                "LDLC ASVEL Villeurbanne",
                "Maccabi Playtika Tel Aviv",
                "Partizan Mozzart Bet Belgrade",
                "Anadolu Efes Istanbul",
                "FC Barcelona",
                "ALBA Berlin",
                "FC Bayern Munich",
                "Virtus Segafredo Bologna",
                "Zalgiris Kaunas",
                "EA7 Emporio Armani Milan",
                "Fenerbahce Beko Istanbul",
                "Olympiacos Piraeus",
                "Panathinaikos AKTOR Athens",
                "Baskonia Vitoria-Gasteiz",
                "Real Madrid",
                "AS Monaco",
                "Valencia Basket")
)

print(tag_name_map)

# A tibble: 18 × 2
   `TM TAG` `TM NAME`                         
   <chr>    <chr>                             
 1 CZB      Crvena Zvezda Meridianbet Belgrade
 2 ASV      LDLC ASVEL Villeurbanne           
 3 MTA      Maccabi Playtika Tel Aviv         
 4 PAR      Partizan Mozzart Bet Belgrade     
 5 EFE      Anadolu Efes Istanbul             
 6 FCB      FC Barcelona                      
 7 BER      ALBA Berlin                       
 8 BAY      FC Bayern Munich                  
 9 VIR      Virtus Segafredo Bologna          
10 ZAL      Zalgiris Kaunas                   
11 MIL      EA7 Emporio Armani Milan          
12 FEN      Fenerbahce Beko Istanbul          
13 OLY      Olympiacos Piraeus                
14 PAN      Panathinaikos AKTOR Athens        
15 BAS      Baskonia Vitoria-Gasteiz          
16 RMB      Real Madrid                       
17 ASM      AS Monaco                         
18 VAL      Valencia Basket                   


In [43]:
clean_team_names <- function(teams, tag_map) {
  teams %>%
    left_join(tag_map, by = "TM TAG") %>%   # Dopasowanie poprawnej nazwy
    filter(`TM NAME.x` == `TM NAME.y`) %>%  # Zachowanie tylko poprawnych nazw
    select(-`TM NAME.y`) %>%                # Usuwanie pomocniczej kolumny
    rename(`TM NAME` = `TM NAME.x`) #%>%     # Przywracanie oryginalnej nazwy
    #mutate(`MIN` = as.integer(`MIN` / 5))   # Podzielenie liczby min przez 5
}


In [44]:
scale_minutes <- function(teams) {
  teams %>%
    mutate(`MIN` = as.integer(`MIN` / 5))   # Podzielenie liczby min przez 5
}

In [45]:
teams_regular <- scale_minutes(clean_team_names(teams_regular, tag_name_map))
teams_playin <- scale_minutes(clean_team_names(teams_playin, tag_name_map))
teams_playoff <- scale_minutes(clean_team_names(teams_playoff, tag_name_map))
teams_final4 <- scale_minutes(clean_team_names(teams_final4, tag_name_map))

In [46]:
print(teams_playin)

# A tibble: 6 × 133
  ROUND `TM TAG` `OPP TAG` `TM NAME`    `OPP NAME` DATE  `H/A` `W/L`   MIN   PTS
  <int> <chr>    <chr>     <chr>        <chr>      <chr> <chr> <chr> <int> <int>
1    35 BAS      MTA       Baskonia Vi… Maccabi P… 16.0… A     L        40    85
2    35 MTA      BAS       Maccabi Pla… Baskonia … 16.0… H     W        40   113
3    35 EFE      VIR       Anadolu Efe… Virtus Se… 16.0… H     L        40    64
4    35 VIR      EFE       Virtus Sega… Anadolu E… 16.0… A     W        40    67
5    36 BAS      VIR       Baskonia Vi… Virtus Se… 19.0… H     W        40    89
6    36 VIR      BAS       Virtus Sega… Baskonia … 19.0… A     L        40    77
# ℹ 123 more variables: `2PTM` <int>, `2PTA` <int>, `2PT%` <dbl>, `3PTM` <int>,
#   `3PTA` <int>, `3PT%` <dbl>, FGM <int>, FGA <int>, `FG%` <dbl>, FTM <int>,
#   FTA <int>, `FT%` <dbl>, OR <int>, DR <int>, TR <int>, AST <int>, TO <int>,
#   ST <int>, BLK <int>, BLKA <int>, PF <int>, DF <int>, VAL <int>,
#   `+ / -` <int>, POSS <in

**Czyszczenie danych players**

In [47]:
delete_0min <- function(players) {
  players[players$MIN != 0, ]
}

In [48]:
glimpse(players_playin)

Rows: 72
Columns: 85
$ `TM TAG`       <chr> "BAS", "BAS", "BAS", "BAS", "BAS", "BAS", "BAS", "BAS",…
$ NAME           <chr> "Chima Moneke", "Chris Chiozza", "Codi Miller McIntyre"…
$ ROLE           <chr> "PF", "PG", "PG", "F", "PG/SG", "C", "PG", "PF/C", "F",…
$ NAT            <chr> "NGR", "USA", "USA", "ESP", "MKD", "EST", "USA", "USA",…
$ HEIGHT         <chr> "198", "180", "191", "203", "182", "209", "178", "210",…
$ AGE            <int> 29, 29, 30, 31, 35, 28, 25, 31, 23, 25, 26, 27, 30, 28,…
$ ROUND          <int> 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,…
$ `TM NAME`      <chr> "Baskonia Vitoria-Gasteiz", "Baskonia Vitoria-Gasteiz",…
$ `OPP NAME`     <chr> "Maccabi Playtika Tel Aviv", "Maccabi Playtika Tel Aviv…
$ DATE           <chr> "16.04.2024", "16.04.2024", "16.04.2024", "16.04.2024",…
$ `H/A`          <chr> "A", "A", "A", "A", "A", "A", "A", "A", "A", "A", "A", …
$ `W/L`          <chr> "L", "L", "L", "L", "L", "L", "L", "L", "L", "L", "L", …
$ MIN            <i

In [49]:
players_regular <- delete_0min(players_regular)
players_playin <- delete_0min(players_playin)
players_playoff <- delete_0min(players_playoff)
players_final4 <- delete_0min(players_final4)

In [50]:
glimpse(players_playin)

Rows: 65
Columns: 85
$ `TM TAG`       <chr> "BAS", "BAS", "BAS", "BAS", "BAS", "BAS", "BAS", "BAS",…
$ NAME           <chr> "Chima Moneke", "Chris Chiozza", "Codi Miller McIntyre"…
$ ROLE           <chr> "PF", "PG", "PG", "F", "PG/SG", "C", "PG", "PF/C", "F",…
$ NAT            <chr> "NGR", "USA", "USA", "ESP", "MKD", "EST", "USA", "USA",…
$ HEIGHT         <chr> "198", "180", "191", "203", "182", "209", "178", "210",…
$ AGE            <int> 29, 29, 30, 31, 35, 28, 25, 31, 23, 25, 26, 27, 30, 28,…
$ ROUND          <int> 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,…
$ `TM NAME`      <chr> "Baskonia Vitoria-Gasteiz", "Baskonia Vitoria-Gasteiz",…
$ `OPP NAME`     <chr> "Maccabi Playtika Tel Aviv", "Maccabi Playtika Tel Aviv…
$ DATE           <chr> "16.04.2024", "16.04.2024", "16.04.2024", "16.04.2024",…
$ `H/A`          <chr> "A", "A", "A", "A", "A", "A", "A", "A", "A", "A", "A", …
$ `W/L`          <chr> "L", "L", "L", "L", "L", "L", "L", "L", "L", "L", "L", …
$ MIN            <i

**Usunięcie nieużywanych zmiennych** 

In [51]:
rm(players)
rm(teams)

In [52]:
objs <- ls(envir = .GlobalEnv)
sizes <- sapply(objs, function(x) object.size(get(x)))
sizes <- sort(sizes, decreasing = TRUE)
data.frame(Obiekt = objs[names(sizes)], Rozmiar_MB = round(sizes / 1024^2, 2))

,Obiekt,Rozmiar_MB
,<chr>,<dbl>
players_regular,NA,2.53
teams_regular,NA,0.46
convert_commas_to_dots,NA,0.24
players_regular_mean,NA,0.19
players_playoff,NA,0.16
convert_numeric_to_integer,NA,0.14
players_regular_sum,NA,0.12
mean_teams,NA,0.07
players_playoff_mean,NA,0.07


**Sumowanie i uśrednianie danych**

In [53]:
# Sumowanie statystyk zespołów
sum_teams <- function(teams) {
  teams %>%
    group_by(`TM NAME`) %>%
    summarise(across(where(is.numeric) & !c(ROUND), ~sum(.x, na.rm = TRUE))) %>%
    select(where(is.character) | where(is.integer))
}

In [54]:
teams_regular_sum <- sum_teams(teams_regular)
teams_playin_sum <- sum_teams(teams_playin)
teams_playoff_sum <- sum_teams(teams_playoff)
teams_final4_sum <- sum_teams(teams_final4)

In [55]:
#Sumowanie statystyk graczy 
sum_players <- function(players) {
    players %>% 
      group_by(NAME) %>% 
      summarise(
      AGE = first(AGE), # Zachowanie pierwszej wartości wieku
      `TM NAME` = first(`TM NAME`), # zachowanie nazwy zespołu
      GP = n(),  # Dodanie liczby gier
      across(where(is.numeric) &
               !c(AGE) & !c(ROUND), ~sum(.x, na.rm = TRUE)))  %>%
    #pominięcie sumowanych procentów 
    select(where(is.character) | where(is.integer))
}



In [56]:
players_regular_sum <- sum_players(players_regular)
players_playin_sum <- sum_players(players_playin)
players_playoff_sum <- sum_players(players_playoff)
players_final4_sum <- sum_players(players_final4)

In [57]:
glimpse(players_regular_sum)

Rows: 289
Columns: 73
$ NAME           <chr> "Achille Polonara", "Adam Hanga", "Alberto Abalde", "Al…
$ `TM NAME`      <chr> "Virtus Segafredo Bologna", "Crvena Zvezda Meridianbet …
$ AGE            <int> 33, 35, 29, 29, 30, 24, 24, 25, 31, 31, 19, 27, 27, 28,…
$ GP             <int> 19, 26, 22, 33, 24, 23, 31, 32, 22, 30, 1, 33, 10, 22, …
$ MIN            <int> 230, 522, 295, 858, 450, 174, 491, 547, 368, 387, 1, 88…
$ PTS            <int> 55, 207, 79, 442, 245, 68, 172, 127, 145, 158, 3, 346, …
$ `2PTM`         <int> 12, 42, 9, 96, 68, 22, 52, 16, 15, 66, 0, 88, 14, 7, 58…
$ `2PTA`         <int> 26, 89, 18, 163, 101, 35, 81, 34, 25, 107, 1, 175, 21, …
$ `3PTM`         <int> 10, 33, 18, 61, 26, 1, 12, 24, 33, 1, 1, 36, 6, 47, 0, …
$ `3PTA`         <int> 24, 99, 45, 119, 88, 9, 26, 69, 73, 5, 1, 86, 15, 115, …
$ FGM            <int> 22, 75, 27, 157, 94, 23, 64, 40, 48, 67, 1, 124, 20, 54…
$ FGA            <int> 50, 188, 63, 282, 189, 44, 107, 103, 98, 112, 2, 261, 3…
$ FTM            <

In [58]:
# Średnia per 40 min 
# Argument wejściowy to teams_x_sum
mean_teams <- function(teams) {
  teams %>%
    group_by(`TM NAME`) %>%
    summarise(MIN = sum(MIN, na.rm = TRUE),
              across(where(is.numeric) & !c(MIN), ~sum(.x, na.rm = TRUE))) %>%
    mutate(across(where(is.numeric) & !c(MIN), ~.x / MIN * 40)) %>%
    select(-MIN)
}


In [59]:
teams_regular_mean <- mean_teams(teams_regular_sum)
teams_playin_mean <- mean_teams(teams_playin_sum)
teams_playoff_mean <- mean_teams(teams_playoff_sum)
teams_final4_mean <- mean_teams(teams_final4_sum)

In [60]:
glimpse(teams_regular_mean)

Rows: 18
Columns: 78
$ `TM NAME`      <chr> "ALBA Berlin", "AS Monaco", "Anadolu Efes Istanbul", "B…
$ PTS            <dbl> 76.20588, 81.61172, 84.56318, 83.92674, 81.29412, 77.63…
$ `2PTM`         <dbl> 18.85294, 21.77289, 21.22744, 19.45788, 20.00000, 17.95…
$ `2PTA`         <dbl> 37.08824, 41.11355, 37.80505, 35.07692, 36.44118, 32.58…
$ `3PTM`         <dbl> 8.882353, 7.882784, 9.790614, 10.344322, 9.235294, 9.95…
$ `3PTA`         <dbl> 25.29412, 22.35897, 26.31047, 28.21978, 27.05882, 27.15…
$ FGM            <dbl> 27.73529, 29.65568, 31.01805, 29.80220, 29.23529, 27.91…
$ FGA            <dbl> 62.38235, 63.47253, 64.11552, 63.29670, 63.50000, 59.73…
$ FTM            <dbl> 11.85294, 14.41758, 12.73646, 13.97802, 13.58824, 11.85…
$ FTA            <dbl> 14.97059, 19.45788, 15.65343, 18.34432, 17.64706, 15.27…
$ OR             <dbl> 10.823529, 11.223443, 9.963899, 10.285714, 11.852941, 8…
$ DR             <dbl> 21.61765, 22.91575, 21.77617, 24.82051, 22.85294, 23.09…
$ TR             <d

In [68]:
# Średnia per games played
# Argument wejściowy to players_x_sum
mean_players <- function(players) {
  players %>%
    mutate(across(where(is.numeric) & !c(MIN, AGE, GP), ~.x / GP)) %>%
    select(-MIN)
}

In [69]:
players_regular_mean <- mean_players(players_regular_sum)
players_playin_mean <- mean_players(players_playin_sum)
players_playoff_mean <- mean_players(players_playoff_sum)
players_final4_mean <- mean_players(players_final4_sum)

In [70]:
glimpse(players_playin_mean)

Rows: 44
Columns: 72
$ NAME           <chr> "Achille Polonara", "Alessandro Pajola", "Ante Zizic", …
$ `TM NAME`      <chr> "Virtus Segafredo Bologna", "Virtus Segafredo Bologna",…
$ AGE            <int> 33, 25, 27, 30, 31, 28, 38, 29, 29, 30, 31, 37, 25, 29,…
$ GP             <int> 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2…
$ PTS            <dbl> 5.5, 2.0, 4.0, 11.0, 9.5, 14.0, 0.0, 0.0, 0.0, 20.0, 0.…
$ `2PTM`         <dbl> 0.0, 0.0, 2.0, 1.0, 2.0, 4.0, 0.0, 0.0, 0.0, 7.5, 0.0, …
$ `2PTA`         <dbl> 1.0, 0.5, 3.5, 3.0, 2.0, 8.0, 0.5, 1.0, 0.0, 10.5, 0.0,…
$ `3PTM`         <dbl> 1.5, 0.5, 0.0, 2.0, 1.5, 2.0, 0.0, 0.0, 0.0, 1.0, 0.0, …
$ `3PTA`         <dbl> 2.5, 1.0, 0.0, 3.0, 5.5, 4.0, 0.0, 1.0, 1.0, 3.0, 0.5, …
$ FGM            <dbl> 1.5, 0.5, 2.0, 3.0, 3.5, 6.0, 0.0, 0.0, 0.0, 8.5, 0.0, …
$ FGA            <dbl> 3.5, 1.5, 3.5, 6.0, 7.5, 12.0, 0.5, 2.0, 1.0, 13.5, 0.5…
$ FTM            <dbl> 1.0, 0.5, 0.0, 3.0, 1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, …
$ FTA            <d